# Titanic -- Analyzing Raw Data to Verify Popular Myth

A popular myth surrounding the sinking of the Titanic that the crew observed the Birkenhead Drill, directing women and children to exit the limited life rafts first.  Can this be verified in the data?

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import statsmodels.api as sm
from plotly.tools import FigureFactory as FF
from plotly.tools import make_subplots

titanic_df = pd.read_csv('titanic_data.csv')

First, an overview of what is present in this data set:

In [2]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


For a general idea of what data is present and the data distribution, these are the basic stats on the overall data.  

In [4]:
#Survived is a binary, mapping it to a string for display purposes
titanic_df['Survival'] = titanic_df['Survived'].replace({0:'Victim', 1: 'Survivor'})

#Because of NaN in data, this may show a warning on some versions of Pandas that NaN is present in data
print titanic_df.describe()

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000         NaN    0.000000   
50%     446.000000    0.000000    3.000000         NaN    0.000000   
75%     668.500000    1.000000    3.000000         NaN    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


Out of the approximately 1,317 people on the Titanic at the time of the accident, the Kaggle data used in this project contains 891 people.  This is 67.65% of the passengers.  Out of these, 342 were survivors.  Surprisingly, this means that out of the data available in Kaggle, 38% were survivors.  However, the estimated passenger counts show 53% of the passengers survived.

Because the data set is incomplete in relation to passenger ages, all analysis regarding age will be on the set of data where age is present.

A more detailed distribution of the categorical data can be seen in the following pie charts.

In [5]:
# print titanic_df.Sex.value_counts()
#create dataframe with count of passengers by sex
t_sexvc_df = titanic_df.Sex.value_counts().to_frame()

#label by index (Male/Female for Sex), values are the value counts
sexlabels =t_sexvc_df.index.tolist()
sexvalues = values=t_sexvc_df['Sex'].tolist()

#dataframe and label/values for Survived
t_srv_df = titanic_df.Survival.value_counts().to_frame()
srvlabels = t_srv_df.index.tolist()
srvvalues = t_srv_df['Survival'].tolist()

#dataframe and label/values for Passenger Class
t_pclass_df = titanic_df.Pclass.value_counts().to_frame()
pclasslabels = t_pclass_df.index.tolist()
pclassvalues = t_pclass_df['Pclass'].tolist()

#dataframe and label/values for Cabin
t_emb_df = titanic_df.Embarked.value_counts().to_frame()
emblabels = t_emb_df.index.tolist()
embvalues = t_emb_df['Embarked'].tolist()

fig = {
    'data': [
        {
            'labels': sexlabels,
            'values': sexvalues,
            'type': 'pie',
            'name': 'Sex',
            'hoverinfo': 'label+percent+name',
            'domain': {'x': [0, .48],
                       'y': [.51, 1]},     
        },
        {
            'labels': srvlabels,
            'values': srvvalues,
            'type': 'pie',
            'name': 'Survival',
            'hoverinfo': 'label+percent+name',
            'domain': {'x': [.52, 1],
                       'y': [.51, 1]}
        },
        {
            'labels': pclasslabels,
            'values': pclassvalues,
            'type': 'pie',
            'name': 'Passenger Class',
            'hoverinfo': 'label+percent+name',
            'domain': {'x': [0, .48],
                       'y': [0, .49]}
        },
        {
            'labels': emblabels,
            'values': embvalues,
            'type': 'pie',
            'name': 'Embarked',
            'hoverinfo': 'label+percent+name',
            'domain': {'x': [.52, 1],
                       'y': [0, .49]},
        }
    ],
    'layout': {'title': 'Passenger Totals by Category',
               'showlegend': False,
#                'subplot_titles': {'Sex','Passenger Class'}
              }
}

py.iplot(fig)

# titanic_df['Sex'].iplot()

To look at the distribution for fares, we can use a Violin chart to see the IQR, max, mean, and mininum, as well as a visual representation of the poisson distribution.

In [7]:
vfig = FF.create_violin(titanic_df['Fare'])
py.iplot(vfig, filename = 'One Violin')

The overall distribution for age can be visualized by a histogram.

In [8]:
#note: mouseover has average age for that bin in grey, total passengers in that bin in red.
titanic_df['Age'].iplot(kind='histogram', filename='cufflinks/customized-histogram',
                       yTitle = 'Number of Passengers', xTitle = 'Age',
                       title = 'Age of Passengers', histfunc = 'count'
                       ,bins = 25, text = 'Age')


## Gender

To test whether there is a correlation between Titanic passenger gender and survival, $\chi^2$ is appropriate as gender is a nominal variable.

$H_{0}:\mu_{f}=\mu_{m} $

The null hypothesis for the correlation between gender and survival, $H_{0}$, is there will be no difference between survival rates for passengers based on gender.

$H_{s}: \mu_{f}>\mu_{m}$

The alternative hypothesis, $H_{s}$, is there will be a higher survival rate for female passengers than for male passengers. 

To prove the null hypothesis, a mininum 95% probability or 0.05 p-value threshold will be held.

In [9]:
#create a frequency distribution table to analyze sex:survival
sex_survival_tab = pd.crosstab(titanic_df.Survival, titanic_df.Sex)
print "The actual frequency distribution table for sex and survival is:"
print sex_survival_tab

#calculate chi squared to test for a correlation between sex and survival:
chi2, p, ddof, expected = chi2_contingency(sex_survival_tab)
msg = """\nAt {} degree of freedom, chi squared with Yates' correction for continuity is {}. 
The p-value for this outcome is {}.
\nThe expected values if there were no correlation between sex and survival, also known as the expected bivariate 
distribution, is:\n{}"""
print msg.format(ddof,chi2, p, expected)

The actual frequency distribution table for sex and survival is:
Sex       female  male
Survival              
Survivor     233   109
Victim        81   468

At 1 degree of freedom, chi squared with Yates' correction for continuity is 260.717020167. 
The p-value for this outcome is 1.19735706278e-58.

The expected values if there were no correlation between sex and survival, also known as the expected bivariate 
distribution, is:
[[ 120.52525253  221.47474747]
 [ 193.47474747  355.52525253]]


The value for $\chi^2$ is high, this suggests a correlation between gender and survival.  

$\phi$ will measure the the strength of the relationship between Titanic passenger gender and survival.  

In [10]:
#We can use the uncorrected chi squared to determine phi squared
phi = np.sqrt(chi2_contingency(sex_survival_tab, correction=False)[0]/titanic_df.PassengerId.count())
msg = 'The phi measure of association between sex and survival is {}.'
print msg.format(phi)

The phi measure of association between sex and survival is 0.543351380658.


The P value, 1.19735706278e-58, for this correlation between gender and survival is extremely statistically significant.  

Visually, the relationship between gender and survival is also very strong.

In [11]:
pd.crosstab(titanic_df.Sex, titanic_df.Survival).iplot(kind='bar', filename='cufflinks/bar-chart', title = 'Survival by Gender',
                       yTitle = 'Number of Passengers', xTitle = 'Passenger Gender')

The above graph visually displays how female passengers were significantly more likely to survive than male passengers.

## Age


$H_{0}:\mu_{a}=\mu_{y} $

The null hypothesis, $H_{0}$, is there will be no difference between survival rates for passengers based on age.

$H_{a}: \mu_{a}<\mu_{y}$

The alternative hypothesis, $H_{a}$, is there will be a higher survival rate for younger passengers than for adult passengers. 

To prove the null hypothesis, a mininum 95% probability or 0.05 p-value threshold will be held.

Because the age data is incomplete, we'll limit the data to what contains age and compare that to survival:

In [13]:
#add a binary gender for line purposes
titanic_df['SexBinary']=titanic_df['Sex'].replace({'male': 1, 'female': 0})

#filter data to non-NaN age data
titanic_with_ages_df = titanic_df[titanic_df['Age'].notnull()]

Because age is an ordinal variable, the Mann-Whitney test is an appropriate measurement of the probability that an observation of the survival rate of the population of a certain age would be equal to the survival rate of the general population.

In [14]:
#calculate Mann-Whitney for Age:Survived correlation
print mannwhitneyu(titanic_df['Age'], titanic_df['Survived'], alternative='two-sided')

MannwhitneyuResult(statistic=790290.0, pvalue=4.2195654324180847e-298)


The calculated p-value of 0 is significantly lower than the required p value of 0.05.  Therefore, the Mann-Whitney test shows a very low probability for the age:survival correlation, if age had no effect on survival.  

This distinction is also dramatic visually.

In [15]:
#Create grouped data to display the number of Survivors/Victims by Age
titanic_groupby_age_survived = titanic_with_ages_df.groupby(['Age','Survival'])

#unstack the data so it will show in 'two' stacked graphs
titanic_survival_age = titanic_groupby_age_survived['PassengerId'].count().unstack().fillna(0)

In [27]:
titanic_survival_age.iplot(kind='histogram', filename='cufflinks/customized-histogram',
                           yTitle = 'Number of Passengers', xTitle = 'Age of Passengers',
                           title='Age and Survival for Titanic Passengers',
                           histfunc = 'count'
                          )

## Age and Sex

Age and sex individually appear to have some affect on the survival chances for an individual passenger.  

Let's look at a graph of survival by age and sex combined.

In [28]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('Age and Survival for Female Titanic Passengers',
                                                                       'Age and Survival for Male Titanic Passengers'))

titanic_w_ages_f_df = pd.crosstab(titanic_with_ages_df[titanic_with_ages_df['Sex'] == 'female'].Age, 
                                  titanic_with_ages_df[titanic_with_ages_df['Sex'] == 'female'].Survival)

for col in ['Survivor','Victim']:
    fig.append_trace({'x': titanic_w_ages_f_df.index, 'y': titanic_w_ages_f_df[col], 'type': 'scatter', 'name': 'Female ' + col},
                     1, 1)

titanic_w_ages_m_df = pd.crosstab(titanic_with_ages_df[titanic_with_ages_df['Sex'] == 'male'].Age, 
                                  titanic_with_ages_df[titanic_with_ages_df['Sex'] == 'male'].Survival)

for col in ['Survivor','Victim']:
    fig.append_trace({'x': titanic_w_ages_m_df.index, 'y': titanic_w_ages_m_df[col], 'type': 'scatter', 'name': 'Male ' + col},
                     2, 1) 

fig['layout'].update(title='Survival by Age and Gender')
    
py.iplot(fig, filename='pandas/cufflinks-subplot rows',)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



Age and Survival for Titanic Passengers appears to show a likelihood for victims to be adults rather than seniors or small children.  This supports the theory that the Burkenhead drill was followed; there is a strong correlation between sex and survival, and a statistically significant correlation between age and survival.  

However, before assuming causality, other factors would need to be brought into consideration to ensure that these two correlations are not a result of a common-causal variable.

For example, we will compare age, sex, and passenger class to the survival of each passenger who has a reported age.  Perhaps passenger survival was akin to the blockbuster movie Titanic, and the wealthier passengers had a better rate at survival than the lower-class passengers.

$H_{0}: \mu_{uc}=\mu_{lc}$

Taking the passenger's class into account, a new null hypothesis will be that the upper-class passengers, $\mu_{uc}$, would have the same survival rate as the lower-class passengers, $\mu_{lc}$, while accounting for passenger age and gender.

$H_{c}: \mu_{uc}>\mu_{lc}$

The alternative hypothesis, $\mu_{c}$, is that the survival rate of upper-class passengers is higher than the survival rate of lower-class passengers.

The p-value to prove the null hypothesis will hold at 0.05.

For this hypothesis, the dependent variable is survival and the independent variables are age, sex, and passenger class.

Because survival is a categorical variable, logistic regression is an appopriate test to determine whether the independent variables of age, sex, and passenger class, had a statistically significant influence on the dependent variable of passenger survival.

In [29]:
logit = sm.Logit(titanic_with_ages_df['Survived'], titanic_with_ages_df[['Age', 'SexBinary','Pclass']])
result = logit.fit()
print result.summary()

Optimization terminated successfully.
         Current function value: 0.548600
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      711
Method:                           MLE   Df Model:                            2
Date:                Wed, 26 Oct 2016   Pseudo R-squ.:                  0.1878
Time:                        00:44:20   Log-Likelihood:                -391.70
converged:                       True   LL-Null:                       -482.26
                                        LLR p-value:                 4.691e-40
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Age            0.0242      0.005      5.335      0.000         0.015     0.033
SexBinary     -2.0429      0.

The probability of the survival outcomes if age and sex had no effect are below .0005 and are therefore extremely statistically significant.  However, the probabilty of obtaining this survival rate if passenger class had no effect is .5 and therefore not statistically significant.  By this measure, it is highly unlikely that passenger class was a common-causal variable for the correlations between sex, age, and survival.  

There are many measures in this data, and there are other variables that could be a common-causal variable to explain the correlations shown above.  For example, passengers who were physically located further away from life rafts at the time of the call to evacuate the ship could have had a lower chance of survival due to the limited rafts.  Although we may never know exactly what would have caused any given passenger to survive that evening, the data does provide an interesting look into the effect maritime customs would have on survival rates for passengers.

## References

Chi Square Independence Test for Two Pandas DF columns. Retrieved September 18, 2016, from http://codereview.stackexchange.com/questions/96761/chi-square-independence-test-for-two-pandas-df-columns

Cufflinks. (n.d.). Retrieved September 13, 2016, from https://plot.ly/ipython-notebooks/cufflinks/

Grasmick, H. G. (2005, Fall). Social Statistics. Lecture presented in OK, Norman.

Is there a pythonic way to do a contingency table in Pandas? Retrieved September 17, 2016, from http://stackoverflow.com/questions/29901436/is-there-a-pythonic-way-to-do-a-contingency-table-in-pandas

Pandas: How to use apply function to multiple columns. Retrieved September 13, 2016, from http://stackoverflow.com/questions/16353729/pandas-how-to-use-apply-function-to-multiple-columns

Plot different DataFrames in the same figure. Retrieved September 14, 2016, from http://stackoverflow.com/questions/13872533/plot-different-dataframes-in-the-same-figure

Pearson product-moment correlation coefficient. (n.d.). Retrieved September 13, 2016, from https://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient
    
Plotly. (n.d.). Retrieved September 14, 2016, from https://plot.ly/pandas/
    
Titanic: Machine Learning from Disaster. (n.d.). Retrieved September 13, 2016, from https://www.kaggle.com/c/titanic/data
        
Window, B. &. (n.d.). Pandas.Series.notnull¶. Retrieved September 13, 2016, from http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.notnull.html
    
Women and children first. (n.d.). Retrieved September 13, 2016, from https://en.wikipedia.org/wiki/Women_and_children_first

ŷhat | Logistic Regression in Python Using Rodeo. Retrieved September 18, 2016, from http://blog.yhat.com/posts/logistic-regression-python-rodeo.html